<h1> <b>Analisis Sentimental en los Medios de Comunicacion Colombianos para la Evaluacion de Tendencias Politicas Nacionales<b> <h1>

---

Julian D. Osorio

# Preliminares

In [ ]:
! pip install gnewsclient
! pip install python-Levenshtein
! pip install html2text
! pip install nltk
! pip install flair
! pip install sacremoses
! pip install sentence-transformers
! pip install translate

In [ ]:
# Cliente a GoogleNews
from gnewsclient import gnewsclient
# Abrir conexiones de una página web
import requests
# WebScraping
from bs4 import BeautifulSoup
# Expresiones regulares
import re
# Transformar html a texto
import html2text

# modelos
import nltk
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline
from flair.data import Sentence
from flair.models import TextClassifier
from transformers import pipeline
from translate import Translator

# visualizacion
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# evaluar tiempo
import time as t

In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

# 1. Extracción y Procesamiento de Noticias.

In [ ]:
'''
Posibles palabras clave

Política, Gobierno, Presidente, Ministro, Senado, Congreso, Partido, Elecciones, Reforma, Ley,
Constitución, Democracia, Corrupción, Protesta, Manifestación, Voto, Campaña, Candidato, Derechos,
Política exterior, Política interna, Economía, Inflación, Desempleo, Impuestos, Salud, Educación, Seguridad
'''

client = gnewsclient.NewsClient(location='Colombia', language='spanish', topic='Nation', use_opengraph=True, max_results = 500)
noticias = client.get_news()

p_clave = ['presidente', 'gobierno', 'constituyente', 'petro', 'mandatario', 'reforma', 'ley', 'corrupción']
noticias = [ noticia for noticia in noticias if any( palabra.lower() in noticia['title'].lower() for palabra in p_clave ) ]

# medios_principales = ['el tiempo', 'semana', 'caracol', 'rcn', 'infobae', 'ELESPECTADOR.COM']
# noticias = [ noticia for noticia in noticias if any( medio.lower() in noticia['site_name'].lower() for medio in medios_principales ) ]

In [ ]:
# Ejemplo Noticia Extraida
ejemplo = noticias[-1]

titular = ejemplo['title']
medio = ejemplo['site_name']
url = ejemplo['url']
# puede darse url = None
if url == None or re.search('http', url) == None:
  url = ejemplo['link'] # ir desde Google

print(titular, medio, url, sep = '\n')

Presidente Petro descarta modificar regla fiscal y le apuesta a las inversiones forzosas para reactivar la economía - El Tiempo
El Tiempo
https://www.eltiempo.com/politica/gobierno/presidente-petro-descarta-modificar-regla-fiscal-y-le-apuesta-a-las-inversiones-forzosas-para-reactivar-la-economia-3352686


In [ ]:
def get_cuerpo(noticias):

  cuerpo = {}
  for noticia in noticias:
    cuerpo[ noticia['site_name'] ] = {'encabezados' : [], 'textos' : []}

  for noticia in noticias:
    encabezado = noticia['title']
    url = noticia['url']
    # puede darse url = None
    if url == None or re.search('http', url) == None:
      url = noticia['link'] # ir desde Google

    # Romper defensa dummy anti-robot
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

    # Obtener html de la noticia
    res = requests.get(url, headers = headers)
    html_page = res.content

    soup = BeautifulSoup(html_page, 'html.parser') # html parsing

    texto = []
    if re.search('403', str(res)):
      print('No se puede mirar la Noticia. \nEstado de la petición:', res)

    # El Tiempo
    elif re.search(re.compile('Tiempo|ElTiempo'), encabezado):
      texto = soup.find_all('p',{'class': 'contenido'})

    # Caracol
    elif re.search(re.compile('Caracol'), encabezado):
      texto = soup.find_all('p')
      # las noticias de caracol comienzan con negrita
      p = re.compile('^[^\*]*\*')
      # Quitar Las propagandas del inicio
      texto = re.sub(p,'',''.join(str(texto)))

    # Semana, Infobae y Otros
    else:
      texto = soup.find_all( lambda tag: tag.name == 'p' and ( 'paragraph' in tag.get('class', []) or tag.get('data-type') == 'text' ) )

    texto_limpio = []
    if texto:
      # Convertit HTML a Texto
      texto_limpio = html2text.html2text(''.join(str(texto)))

      #Quitar links
      texto_limpio = re.sub(r'\([^)]*\)',' ',texto_limpio)

      # Quitar paréntesis redondos y cuadrados después de quitar links
      texto_limpio = re.sub(r'[\[\]\(\)]','',texto_limpio)

      # Quitar saltos de línea
      texto_limpio = re.sub(r'\n',' ',texto_limpio)

      # Quitar asteriscos
      texto_limpio = re.sub(r'\*',' ',texto_limpio)

      # Quitar comillas sencillas y reemplazar por dobles (los modelos las leen mejor)
      texto_limpio = re.sub(r"[\\']",'\'',texto_limpio)

      #Quitar todo lo que no tenga punto al final, después del punto final.
      texto_limpio = re.sub(r'([^.]*$)','',texto_limpio)

      # Quitar Espacios extra (dos o más)
      texto_limpio = re.sub('\s\s+', ' ',texto_limpio)

      # Quitar el patrón . , texto
      texto_limpio = re.sub('\.\s,\s', '. ',texto_limpio)

      # Quitar espacios iniciales y finales
      texto_limpio = texto_limpio.strip()

      cuerpo[ noticia['site_name'] ]['textos'].append(texto_limpio)
      cuerpo[ noticia['site_name'] ]['encabezados'].append( encabezado.split('-')[0] )


  vacias = []
  for key in cuerpo.keys():
    if len( cuerpo[key]['encabezados'] ) == 0 or len( cuerpo[key]['textos'] ) == 0 :
      vacias.append(key)
  for key in vacias : del cuerpo[key]

  return cuerpo

In [ ]:
cuerpo = get_cuerpo(noticias)

In [ ]:
for key in cuerpo.keys():
  print(key)
  print( cuerpo[key]['encabezados'][0] )
  print( cuerpo[key]['textos'][0][:125], '...' )

  print()

Semana.com   Últimas Noticias de Colombia y el Mundo
Reforma a la educación acorrala al Gobierno Petro: el proyecto desató la furia de Fecode y molestias en el Pacto Histórico 
Una negociaci√≥n liderada en el Congreso por la propia ministra de Educaci√≥n, Aurora Vergara Figueroa, tiene a los maestros  ...

Caracol Radio
Petro cuestiona al Congreso por eliminar derechos sindicales en la Reforma Laboral 
Actualizado 16 Jun 2024 02:58 , Deportes , Ciclismo , Ciclismo , Fútbol , Fútbol , Actualidad , Tendencias , Tiempo libre , M ...



# Modelamiento

In [ ]:
# pequeña funcion auxiliar para imprimir diccionarios
def display(dic):
  for key in dic.keys():
    print(key); print(dic[key], '\n')

# funcion auxiliar para traducir
def traducir(cuerpo, rapido = False):

  if rapido:
    traductor = Translator(from_lang='es', to_lang='en')

    for key in cuerpo.keys():
      for sub_key in cuerpo[key].keys():

        for i in range( len(cuerpo[key][sub_key]) ):
          texto = cuerpo[key][sub_key][i]
          segmentos = [texto[i:i+500] for i in range(0, len(texto), 500)]

          traduccion = ''.join( traductor.translate(segmento) for segmento in segmentos )

          cuerpo[key][sub_key][i] = traduccion

  else:
    traductor = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")

    for key in cuerpo.keys():
      for sub_key in cuerpo[key].keys():

        for i in range( len(cuerpo[key][sub_key]) ):
          texto = cuerpo[key][sub_key][i]
          segmentos = [texto[i:i+500] for i in range(0, len(texto), 500)]

          traduccion_seg  = traductor(segmentos)
          traduccion = ''.join( [traduccion['translation_text'] for traduccion in traduccion_seg] )

          cuerpo[key][sub_key][i] = traduccion

  return cuerpo

In [ ]:
cuerpo = traducir(cuerpo, True)

In [ ]:
for key in cuerpo.keys():
  print(key)
  print( cuerpo[key]['encabezados'][0] )
  print( cuerpo[key]['textos'][0][:125], '...' )

  print()

Semana.com   Últimas Noticias de Colombia y el Mundo
Education reform encircles the Petro Government: the project unleashed the fury of Fecode and discomfort in the Historical Pact
A negotiation led in Congress by the Minister of Education herself, Aurora Vergara Figueroa, has the teachers in an indefinit ...

Caracol Radio
Petro questions Congress for eliminating union rights in Labor Reform
Updated 16 Jun 2024 02:58 , Sports , Cycling , Cycling , Football , Soccer , News , Trends , Leisure , Motor , Life , Technol ...



### TextBlob

Usamos TextBlob para determinar el grado de subjetividad en la redacción de la Noticia. Entre más cercano a **0** sea un texto, más objetivo este sera; a su vez, si la subjetividad equivale a **1**, significa que el texto es muy subjetivo.

In [ ]:
def subjetividad(cuerpo):
  subs = {}
  for key in cuerpo.keys():
    subs[key] = {'encabezados' : 0, 'textos' : 0}

    sub_encabezados = []
    for encabezado in cuerpo[key]['encabezados']:
      blob = TextBlob(encabezado)

      sub = blob.sentiment.subjectivity
      sub_encabezados.append(sub)

    sub_textos = []
    for texto in cuerpo[key]['textos']:
      blob = TextBlob(texto)

      sub = blob.sentiment.subjectivity
      sub_textos.append(sub)


    subs[key]['textos'] = round( sum(sub_textos) / len(sub_textos), 4 )
    subs[key]['encabezados'] = round( sum(sub_encabezados) / len(sub_encabezados), 4 )

  return subs

def polaridad_blob(cuerpo):
  p_blob = {}
  for key in cuerpo.keys():
    p_blob[key] = {'encabezados' : 0, 'textos' : 0}

    p_encbz = []
    for encabezado in cuerpo[key]['encabezados']:
      blob = TextBlob(encabezado)

      p = blob.sentiment.polarity
      p_encbz.append(p)

    p_txt = []
    for texto in cuerpo[key]['textos']:
      blob = TextBlob(texto)

      p = blob.sentiment.polarity
      p_txt.append(p)


    p_blob[key]['textos'] = round( sum(p_txt) / len(p_txt), 4 )
    p_blob[key]['encabezados'] = round( sum(p_encbz) / len(p_encbz), 4 )

  return p_blob

In [ ]:
t1 = t.time()
subjs = subjetividad(cuerpo)
t2 = t.time()

subjs_t = round(t2 - t1, 3)

display(subjs)
print( f'\nTiempo requerido: {subjs_t} s')

Semana.com   Últimas Noticias de Colombia y el Mundo
{'encabezados': 0.0, 'textos': 0.4365} 

Caracol Radio
{'encabezados': 0.0, 'textos': 0.4795} 


Tiempo requerido: 0.066 s


In [ ]:
t1 = t.time()
p_blob = polaridad_blob(cuerpo)
t2 = t.time()

blob_t = round(t2 - t1, 3)

display(p_blob)
print( f'\nTiempo requerido: {blob_t} s')

Semana.com   Últimas Noticias de Colombia y el Mundo
{'encabezados': 0.0, 'textos': 0.0938} 

Caracol Radio
{'encabezados': 0.0, 'textos': 0.1044} 


Tiempo requerido: 0.014 s


Calcularemos la polaridad de las noticias, donde **-1** representa un texto mas negativo, **0** un texto neutral y **1** uno mas positivo.

### VADER


In [ ]:
def polaridad_vader(cuerpo):
  sia = SentimentIntensityAnalyzer()

  p_vader = {}
  for key in cuerpo.keys():
    p_vader[key] = {'encabezados' : 0, 'textos' : 0}

    p_encbz = []
    for encabezado in cuerpo[key]['encabezados']:
      polaridad = sia.polarity_scores(encabezado)
      p_encbz.append( polaridad['compound'] )

    p_txt = []
    for texto in cuerpo[key]['textos']:
      polaridad = sia.polarity_scores(texto)
      p_txt.append( polaridad['compound'] )

    p_vader[key]['textos'] = round( sum(p_txt) / len(p_txt), 4 )
    p_vader[key]['encabezados'] = round( sum(p_encbz) / len(p_encbz), 4 )

  return p_vader

In [ ]:
t1 = t.time()
p_vader = polaridad_vader(cuerpo)
t2 = t.time()

vader_t = round(t2 - t1, 3)

display(p_vader)
print( f'\nTiempo requerido: {vader_t} s')

Semana.com   Últimas Noticias de Colombia y el Mundo
{'encabezados': -0.7579, 'textos': 0.9976} 

Caracol Radio
{'encabezados': 0.0, 'textos': 0.9577} 


Tiempo requerido: 0.036 s


### BERT

In [ ]:
def polaridad_bert(cuerpo):
  p_bert = {}
  nlp = pipeline('sentiment-analysis')

  for key in cuerpo.keys():
    p_bert[key] = {'encabezados' : 0, 'textos' : 0}

    p_encbz = []
    for encabezado in cuerpo[key]['encabezados']:
      # Dividir el encabezado en fragmentos de 512 tokens
      encabezado_fragmentos = [encabezado[i:i+512] for i in range(0, len(encabezado), 512)]
      for fragmento in encabezado_fragmentos:
        polaridad = nlp(fragmento)
        score = polaridad[0]['score']

        if polaridad[0]['label'] == 'NEGATIVE':
          score *= -1

        p_encbz.append(score)

    p_txt = []
    for texto in cuerpo[key]['textos']:
      texto_fragmentos = [texto[i:i+512] for i in range(0, len(texto), 512)]
      for fragmento in texto_fragmentos:
        polaridad = nlp(fragmento)
        score = polaridad[0]['score']

        if polaridad[0]['label'] == 'NEGATIVE':
          score *= -1

        p_txt.append(score)

    p_bert[key]['textos'] = round(sum(p_txt) / len(p_txt), 4) if p_txt else 0
    p_bert[key]['encabezados'] = round(sum(p_encbz) / len(p_encbz), 4) if p_encbz else 0

  return p_bert

In [ ]:
t1 = t.time()
p_bert = polaridad_bert(cuerpo)
t2 = t.time()

bert_t = round(t2 - t1, 3)

print()
display(p_bert)
print( f'\nTiempo requerido: {bert_t} s')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.




Semana.com   Últimas Noticias de Colombia y el Mundo
{'encabezados': 0.5222, 'textos': -0.8586} 

Caracol Radio
{'encabezados': -0.9314, 'textos': 0.341} 


Tiempo requerido: 4.367 s


### Flair

In [ ]:
def polaridad_flair(cuerpo):

  traductor = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")
  clasificador = TextClassifier.load('en-sentiment')

  p_flair = {}
  for key in cuerpo.keys():
    p_flair[key] = {'encabezados' : 0, 'textos' : 0}

    p_encbz = []
    for encabezado in cuerpo[key]['encabezados']:
      frase = Sentence(encabezado)
      clasificador.predict(frase)

      sentimiento = frase.labels[0].value
      polaridad = frase.labels[0].score
      if sentimiento == 'NEGATIVE' : polaridad *= -1

      p_encbz.append( polaridad )


    p_txt = []
    for texto in cuerpo[key]['textos']:
      frase = Sentence(texto)
      clasificador.predict(frase)

      sentimiento = frase.labels[0].value
      polaridad = frase.labels[0].score
      if sentimiento == 'NEGATIVE' : polaridad *= -1

      p_txt.append( polaridad )

    p_flair[key]['textos'] = round( sum(p_txt) / len(p_txt), 4 )
    p_flair[key]['encabezados'] = round( sum(p_encbz) / len(p_encbz), 4 )

  return p_flair

In [ ]:
t1 = t.time()
p_flair = polaridad_flair(cuerpo)
t2 = t.time()

flair_t = round(t2 - t1, 3)

print()
display(p_flair)
print( f'\nTiempo requerido: {flair_t} s')


Semana.com   Últimas Noticias de Colombia y el Mundo
{'encabezados': 0.8003, 'textos': -0.9979} 

Caracol Radio
{'encabezados': -0.9974, 'textos': 0.9056} 


Tiempo requerido: 7.646 s


# Evaluacion

### Resultados de Tiempo

In [ ]:
time_df = {
    'Blob - Subj' : [subjs_t],
    'Blob - Polrd' : [blob_t],
    'VADER' : [vader_t],
    'BERT' : [bert_t],
    'Flair' : [flair_t],
}

time_df = pd.DataFrame(time_df)
time_df

,Blob - Subj,Blob - Polrd,VADER,BERT,Flair
0,0.066,0.014,0.036,4.367,7.646


In [ ]:
fig = px.bar(time_df, title = 'Tiempo de Ejecución de los Modelos', barmode = 'group', text_auto=True )
fig.update_layout(yaxis_title='Tiempo [s]', xaxis_title = 'Modelo')

fig.show()

### Resultados de Metricas

In [ ]:
resultados = [p_blob, p_vader, p_bert, p_flair]
models = ['Blob', 'VADER', 'BERT', 'Flair']

encbz_df = {
    'Medio' : [],
    'Blob' : [],
    'VADER' : [],
    'BERT' : [],
    'Flair' : [],
}

txt_df = {
    'Medio' : [],
    'Blob' : [],
    'VADER' : [],
    'BERT' : [],
    'Flair' : [],
}

subj_df = {
  'Medio' : [],
  'Encabezados' : [],
  'Textos' : [],
}

for key in cuerpo.keys():
  encbz_df['Medio'].append(key)
  txt_df['Medio'].append(key)

  subj_df['Medio'].append(key)
  subj_df['Encabezados'].append( subjs[key]['encabezados'] )
  subj_df['Textos'].append( subjs[key]['textos'] )

  for dic, model in zip(resultados, models):
    txt_df[model].append( dic[key]['textos'] )
    encbz_df[model].append( dic[key]['encabezados'] )


encbz_df = pd.DataFrame(encbz_df)
txt_df = pd.DataFrame(txt_df)
subj_df = pd.DataFrame(subj_df)

In [ ]:
subj_df

,Medio,Encabezados,Textos
0,Semana.com Últimas Noticias de Colombia y el...,0.0,0.4365
1,Caracol Radio,0.0,0.4795


In [ ]:
encbz_df

,Medio,Blob,VADER,BERT,Flair
0,Semana.com Últimas Noticias de Colombia y el...,0.0,-0.7579,0.5222,0.8003
1,Caracol Radio,0.0,0.0000,-0.9314,-0.9974


In [ ]:
txt_df

,Medio,Blob,VADER,BERT,Flair
0,Semana.com Últimas Noticias de Colombia y el...,0.0938,0.9976,-0.8586,-0.9979
1,Caracol Radio,0.1044,0.9577,0.3410,0.9056


In [ ]:
fig = px.bar(subj_df, x='Medio', y= list(subj_df.columns), title = 'Subjetividad en los Medios', barmode = 'group', text_auto=True )
fig.add_hline(y = 1.05, line = dict(color='crimson', width = 0), annotation_text = "SUBJETIVO", annotation_position = "bottom left", annotation_font=dict(color="crimson") )
fig.update_layout(yaxis_title='Subjetividad')

fig.show()

In [ ]:
fig = px.bar(encbz_df, x='Medio', y= list(encbz_df.columns), title = 'Polaridad en los Encabezados de Noticias', barmode = 'group', text_auto=True )
fig.add_hline(y = 1.15, line = dict(color='forestgreen', width = 0), annotation_text = "POSITIVE", annotation_position = "bottom left", annotation_font=dict(color="forestgreen") )
fig.add_hline(y = -1.15, line = dict(color='crimson', width = 0), annotation_text = "NEGATIVE", annotation_position = "top left", annotation_font=dict(color="crimson") )
fig.update_layout(yaxis_title='Polaridad')

fig.show()

In [ ]:
fig = px.bar(txt_df, x='Medio', y= list(txt_df.columns), title = 'Polaridad en los Textos de Noticias', barmode = 'group', text_auto=True )
fig.add_hline(y = 1.15, line = dict(color='forestgreen', width = 0), annotation_text = "POSITIVE", annotation_position = "bottom left", annotation_font=dict(color="forestgreen") )
fig.add_hline(y = -1.15, line = dict(color='crimson', width = 0), annotation_text = "NEGATIVE", annotation_position = "top left", annotation_font=dict(color="crimson") )
fig.update_layout(yaxis_title='Polaridad')

fig.show()